In [3]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sklearn

In [244]:
patient_df = pd.read_csv('data/allpatients_imputed_df').drop(columns=['Unnamed: 0'])

In [245]:
patient_df.shape

(188453, 42)

In [246]:
patient_df.columns

Index(['pid', 'HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2',
       'BaseExcess', 'HCO3', 'FiO2', 'pH', 'PaCO2', 'SaO2', 'AST', 'BUN',
       'Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct',
       'Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium',
       'Bilirubin_total', 'TroponinI', 'Hct', 'Hgb', 'PTT', 'WBC',
       'Fibrinogen', 'Platelets', 'Age', 'Gender', 'Unit1', 'Unit2',
       'HospAdmTime', 'ICULOS', 'SepsisLabel'],
      dtype='object')

In [247]:
patient_df.head()

,pid,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,...,WBC,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel
0,p01065,83.0,100.0,36.7,103.0,68.0,58.0,10.0,NaN,NaN,...,6.1,NaN,125.0,54,1,1.0,0.0,-5.76,1,0
1,p01065,83.0,100.0,36.7,103.0,68.0,58.0,10.0,NaN,NaN,...,6.1,NaN,125.0,54,1,1.0,0.0,-5.76,2,0
2,p01065,80.0,99.0,36.7,103.0,69.0,58.0,10.0,NaN,NaN,...,6.1,NaN,125.0,54,1,1.0,0.0,-5.76,3,0
3,p01065,87.0,99.0,36.7,107.0,76.0,67.0,10.0,NaN,NaN,...,6.1,NaN,125.0,54,1,1.0,0.0,-5.76,4,0
4,p01065,91.0,99.0,36.7,106.0,76.0,68.0,10.0,NaN,NaN,...,6.1,NaN,125.0,54,1,1.0,0.0,-5.76,5,0


#### Fill in NaN values with mean of the column

In [249]:
patient_df = patient_df.fillna(patient_df.mean()['HR':'Platelets'])

In [274]:
patient_df.to_csv('data/allpatients_imputed_df')

#### Sample 6 time-steps from each patient to create new dataset X

In [251]:
sepsis_patients = patient_df.groupby('pid').filter(lambda x: x['SepsisLabel'].any())
normal_patients = patient_df.groupby('pid').filter(lambda x: x['SepsisLabel'].sum() == 0)

In [259]:
(sepsis_patients.groupby('pid').ICULOS.agg(['count'])['count'] < 14).sum()

76

In [262]:
# For sepsis patients, get the 6 measurements before t_sepsis and create new dataset X_sepsis. 
sepsis_patient_agg = sepsis_patients.groupby('pid')
X_sepsis = pd.DataFrame(data=[])

for pid, data in sepsis_patient_agg:
    data.reset_index(drop=True, inplace=True)
    
    if data.shape[0] >= 13:
        first_sepsis_idx =  data['SepsisLabel'].idxmax()
        start_idx = first_sepsis_idx - 6
        
        if start_idx >= 0:
            new_data = data.iloc[start_idx:first_sepsis_idx]
            assert new_data.shape[0] == 6
            X_sepsis = pd.concat([X_sepsis, new_data])

In [166]:
X_sepsis.shape

(1176, 42)

In [260]:
# For normal patients, randomly sample 6 measurements. 
normal_patient_agg = normal_patients.groupby('pid')
X_normal = pd.DataFrame(data=[])

for pid, data in normal_patient_agg:
    data.reset_index(drop=True, inplace=True)
    rand_idx = np.random.randint(0, high=(data.shape[0] - 6))
    new_data = data.iloc[rand_idx:rand_idx+6]
    X_normal = pd.concat([X_normal, new_data])

In [261]:
X_normal.pid.unique().shape

(4721,)

In [263]:
# Create final dataset where we take each patient's rows and stack them into a single vector. 
X = np.ndarray((X_sepsis.pid.unique().shape[0] + X_normal.pid.unique().shape[0], 228))
y = [1]*(X_sepsis.pid.unique().shape[0]) + [0]*(X_normal.pid.unique().shape[0])

i = 0
for pid, df in X_sepsis.groupby('pid'):
    df = df.drop(columns=['pid', 'Unit1', 'Unit2', 'SepsisLabel'])
    x_i = df.stack().values
    X[i,:] = x_i
    i += 1

for pid, df in X_normal.groupby('pid'):
    df = df.drop(columns=['pid', 'Unit1', 'Unit2', 'SepsisLabel'])
    x_i = df.stack().values
    X[i,:] = x_i
    i += 1

In [264]:
X.shape

(4917, 228)

In [265]:
y = np.array(y)

In [266]:
y.shape

(4917,)

In [267]:
y.sum()

196

In [268]:
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix, precision_score, recall_score
from sklearn.model_selection import train_test_split

In [269]:
X, y = sklearn.utils.shuffle(X, y)

In [270]:
X.shape, y.shape

((4917, 228), (4917,))

In [271]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [272]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((3687, 228), (3687,), (1230, 228), (1230,))

## Regularized Log. Regression

In [242]:
from sklearn.linear_model import LogisticRegressionCV

lr_model = LogisticRegressionCV(cv=5, penalty='l2', max_iter=2000, solver='lbfgs')
lr_model = lr_model.fit(X_train, y_train)

/usr/local/Cellar/python3/3.5.2_3/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/Cellar/python3/3.5.2_3/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/Cellar/python3/3.5.2_3/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/Cellar/python3/3.5.2_3/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iter

/usr/local/Cellar/python3/3.5.2_3/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/Cellar/python3/3.5.2_3/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [273]:
lr_model.score(X_test, y_test)

0.9723577235772358

In [275]:
y_pred = lr_model.predict_proba(X_test)
roc_auc_score(y_test, y_pred[:,1])

0.8456610169491525

In [276]:
y_hat = lr_model.predict(X_test)
f1_score(y_test, y_hat)

0.5277777777777778

In [277]:
confusion_matrix(y_test, y_hat)

array([[1177,    3],
       [  31,   19]])

## SVM

In [278]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

for c in [1, 2, 4, 8, 16]:
    svm_model = SVC(gamma='scale', C=c)
    scores = cross_val_score(svm_model, X_train, y_train, cv=5, scoring='roc_auc')
    avg_score = scores.mean()
    print(c, avg_score)

1 0.7712702508204196
2 0.7711887872850168
4 0.761789889640774
8 0.7579563113425478
16 0.7578881250966877


In [279]:
svm_model = SVC(gamma='scale', C=1, probability=True)
svm_model = svm_model.fit(X_train, y_train)

In [280]:
svm_model.score(X_test, y_test)

0.959349593495935

In [281]:
y_pred = svm_model.predict_proba(X_test)
roc_auc_score(y_test, y_pred[:,1])

0.788728813559322

In [282]:
y_hat = svm_model.predict(X_test)
f1_score(y_test, y_hat)

/usr/local/Cellar/python3/3.5.2_3/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.0

In [283]:
confusion_matrix(y_test, y_hat)

array([[1180,    0],
       [  50,    0]])

## RF

In [284]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier()
rf_model = rf_model.fit(X_train, y_train)

/usr/local/Cellar/python3/3.5.2_3/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [285]:
rf_model.score(X_test, y_test)

0.9723577235772358

In [286]:
y_pred = rf_model.predict_proba(X_test)
roc_auc_score(y_test, y_pred[:,1])

0.7957372881355932

In [287]:
y_hat = rf_model.predict(X_test)
f1_score(y_test, y_hat)

0.5142857142857143